### installs

In [ ]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.8 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=05adc9b38d6a9053645b7c8bc5415d0219d93bfe450815c76182bd36fd87fe31
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


### imports

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import random
import csv
from sentence_transformers import CrossEncoder, SentenceTransformer
import numpy as np
import math
import time
from transformers import pipeline

### setup models

In [ ]:
# Load models
model = CrossEncoder('cross-encoder/stsb-roberta-base')
embedding_model = SentenceTransformer('bert-base-nli-mean-tokens')

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/399 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

### define functions

In [ ]:
def random_consecutive_sentence_similarity(text, samples=10):
    # Split text into sentences
    # sentence_endings = re.compile(r'([.!?](?:\s|$)|\n)')
    sentence_endings = re.compile(r'[.!?](?:\s|$)')
    sentences = re.split(sentence_endings, text)

    # Ensure there are at least two sentences to compare
    if len(sentences) < 2:
        return None

    similarity_scores = []

    for _ in range(samples):
        # Randomly select a starting point for the two consecutive sentences
        start_index = random.randint(0, len(sentences) - 2)
        consecutive_sentences = sentences[start_index:start_index + 2]

        # Calculate similarity using the cross-encoder model
        similarity = model.predict(consecutive_sentences)
        similarity_scores.append(similarity)

    # Calculate the average similarity
    average_similarity = np.mean(similarity_scores)

    return average_similarity

def fetch_html_content(url):
    # Ensure the URL starts with http:// or https://
    if not url.startswith(('http://', 'https://')):
        url = 'https://' + url

    try:
        response = requests.get(url, timeout=10)
        html_content = response.text
        return html_content
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch HTML content for URL: {url}")
        return None


def count_sentences(text):
    # Define a regular expression pattern for sentence endings
    sentence_endings = re.compile(r'[.!?](?:\s|$)')
    # sentence_endings = re.compile(r'([.!?](?:\s|$)|\n)')

    # Find all the sentence endings in the text
    sentence_endings_found = sentence_endings.findall(text)

    # The number of sentences is the number of sentence endings found
    return len(sentence_endings_found)

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def word_count(text):
    # Count words in the text
    words = text.split()
    return len(words)

def embed_text(text, max_length=512):
    # Split the text into manageable parts
    parts = [text[i:i+max_length] for i in range(0, len(text), max_length)]
    embeddings = []

    for part in parts:
        embedding = embedding_model.encode(part)
        embeddings.append(embedding)

    # Combine the embeddings
    combined_embedding = np.mean(embeddings, axis=0)
    return combined_embedding.tolist()


def process_urls(file_path):
    with open(file_path, 'r') as file, open('output.csv', 'w', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        # Write the header
        csv_writer.writerow(['URL', 'Metric', 'Sentence Count', 'Word Count', 'Similarity Score', 'Embedded HTML', 'Embedded Text'])

        urls = [url.strip() for url in file.readlines()]
        for url in urls:
            # print(url)
            html_content = fetch_html_content(url)
            if html_content is not None:
                soup = BeautifulSoup(html_content, 'html.parser')
                text_content = soup.get_text()
                if text_content:
                    similarity_score = random_consecutive_sentence_similarity(text_content)
                    sentence_count = count_sentences(text_content)
                    wc = word_count(text_content)
                    score = sigmoid(sentence_count * similarity_score * 0.01) if similarity_score else None

                    # Directly embed the HTML and text content
                    text_embedding = None #embed_text(text_content)
                    html_embedding = None #embed_text(html_content)

                    # Write row to CSV
                    csv_writer.writerow([url, score, sentence_count, wc, similarity_score, html_embedding, text_embedding])
                    print([url, score, sentence_count, wc, similarity_score, html_embedding, text_embedding])
                else:
                    # Handle no text content
                    csv_writer.writerow([url, None, 0, 0, None, '', ''])
                    print([url, None, 0, 0, None, '', ''])
            else:
                # If failed to fetch HTML content, do not write anything to CSV
                continue

### run


In [ ]:
  file_path = './urls_test.txt'
  start = time.time()
  process_urls(file_path)
  end = time.time()
  print(f"Total Processing Time: {end-start} seconds")